# Module 1.12: Portfolio Architecture — The Strategic View

> **Goal:** Triage the business and identify where risk and investment lives.

In Module 1.10, we computed Structure and Chaos scores for every series. Now we use those coordinates to build a strategic map of our portfolio.

**This module answers:**
- What does our portfolio look like on a Structure × Chaos map?
- Which series are Stable? Complex? Messy? Low Signal?
- Where is our revenue concentrated — and what's the risk profile?
- Which departments need automation vs. human oversight?

| Previous Module (1.10) | This Module (1.12) |
|------------------------|--------------------|
| Compute coordinates | Plot the map |
| Sanity check metrics | Assign archetypes |
| Output scores | Strategic triage |

---

## 1. Setup and Load Data

In [ ]:
# =============================================================================
# SETUP
# =============================================================================

# --- Imports ---
import sys
import warnings
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd

# --- Path Configuration (before local imports) ---
MODULE_DIR = Path().resolve()
PROJECT_ROOT = MODULE_DIR.parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

# --- Local Imports ---
import tsforge as tsf
from src import (
    CacheManager,
    ArtifactManager,
    get_notebook_name
)

# --- Settings ---
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8-whitegrid")

# --- Paths ---
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# --- Managers ---
NB_NAME = get_notebook_name()
cache = CacheManager(PROJECT_ROOT / ".cache" / NB_NAME)
artifacts = ArtifactManager(PROJECT_ROOT / "artifacts")

print(f"✓ Setup complete | Root: {PROJECT_ROOT.name} | Module: {NB_NAME[:4]}")

In [ ]:
# --- Load Scored Data from Module 1.10 ---
scores_df = artifacts.load('1.10')
print(f"Loaded scores for {len(scores_df):,} series")

# --- Load Raw Time Series from Module 1.08 ---
df = artifacts.load('1.08')
print(f"Loaded time series: {len(df):,} rows")

In [ ]:
# Preview scores
scores_df.head()

---

## 2. The Structure × Chaos Map

This is the core visualization: every series plotted by its Structure Score (y-axis) and Chaos Score (x-axis).

**Reading the quadrants:**

| Quadrant | Structure | Chaos | Archetype | Interpretation |
|----------|-----------|-------|-----------|----------------|
| Top-Left | High | Low | **Stable** | Predictable patterns, low noise → automate |
| Top-Right | High | High | **Complex** | Strong patterns but also high noise → advanced models |
| Bottom-Right | Low | High | **Messy** | No patterns, high noise → human oversight |
| Bottom-Left | Low | Low | **Low Signal** | Neither patterns nor noise → simple baselines |

### 2.1 Baseline Method: Visual Split at 0.5

The simplest approach: split at the midpoint (0.5) on each axis.

In [ ]:
# =============================================================================
# THE STRUCTURE × CHAOS MAP
# =============================================================================

fig, ax = plt.subplots(figsize=(10, 10))

# Scatter all series
ax.scatter(
    scores_df['chaos_score'], 
    scores_df['structure_score'],
    alpha=0.3,
    s=10,
    c='#4A4E69',
    edgecolors='none'
)

# Add quadrant lines at 0.5
ax.axhline(y=0.5, color='#E94F37', linestyle='--', linewidth=2, alpha=0.7)
ax.axvline(x=0.5, color='#E94F37', linestyle='--', linewidth=2, alpha=0.7)

# Quadrant labels
label_style = dict(fontsize=14, fontweight='bold', alpha=0.6)
ax.text(0.25, 0.75, 'STABLE', ha='center', va='center', color='#2E86AB', **label_style)
ax.text(0.75, 0.75, 'COMPLEX', ha='center', va='center', color='#8338EC', **label_style)
ax.text(0.75, 0.25, 'MESSY', ha='center', va='center', color='#E94F37', **label_style)
ax.text(0.25, 0.25, 'LOW SIGNAL', ha='center', va='center', color='#6C757D', **label_style)

# Formatting
ax.set_xlabel('Chaos Score →', fontsize=12)
ax.set_ylabel('Structure Score →', fontsize=12)
ax.set_title('Portfolio Architecture: Structure × Chaos Map', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')

# Add count annotations
n_total = len(scores_df)
ax.text(0.02, 0.98, f'n = {n_total:,} series', transform=ax.transAxes, 
        fontsize=10, va='top', color='gray')

plt.tight_layout()
plt.show()

### 2.2 Advanced Method: K-Means Clustering (Optional)

> **Advanced** — This section uses K-Means clustering to find natural groupings in the data. The details of K-Means are beyond our scope; we simply demonstrate that data-driven clustering largely confirms our visual quadrant split.

In [ ]:
# =============================================================================
# ADVANCED: K-MEANS CLUSTERING
# =============================================================================

from sklearn.cluster import KMeans

# Fit K-Means with 4 clusters
X = scores_df[['chaos_score', 'structure_score']].values
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
scores_df['cluster_kmeans'] = kmeans.fit_predict(X)

# Get cluster centers
centers = kmeans.cluster_centers_

print("K-Means Cluster Centers:")
print("-" * 40)
for i, (chaos, structure) in enumerate(centers):
    print(f"  Cluster {i}: Chaos={chaos:.3f}, Structure={structure:.3f}")

In [ ]:
# Plot with K-Means clusters
fig, ax = plt.subplots(figsize=(10, 10))

# Color by cluster
colors = ['#2E86AB', '#8338EC', '#E94F37', '#6C757D']
for cluster_id in range(4):
    mask = scores_df['cluster_kmeans'] == cluster_id
    ax.scatter(
        scores_df.loc[mask, 'chaos_score'],
        scores_df.loc[mask, 'structure_score'],
        alpha=0.3, s=10, c=colors[cluster_id], edgecolors='none',
        label=f'Cluster {cluster_id} (n={mask.sum():,})'
    )

# Plot cluster centers
ax.scatter(centers[:, 0], centers[:, 1], c='black', s=200, marker='X', 
           edgecolors='white', linewidths=2, zorder=5, label='Centroids')

# Reference lines
ax.axhline(y=0.5, color='gray', linestyle=':', linewidth=1, alpha=0.5)
ax.axvline(x=0.5, color='gray', linestyle=':', linewidth=1, alpha=0.5)

ax.set_xlabel('Chaos Score →', fontsize=12)
ax.set_ylabel('Structure Score →', fontsize=12)
ax.set_title('K-Means Clustering (k=4) vs. Visual Quadrants', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.legend(loc='upper left', fontsize=9)

plt.tight_layout()
plt.show()

---

## 3. Assign Archetypes

Using the baseline 0.5/0.5 split, we assign each series to one of four archetypes.

In [ ]:
# =============================================================================
# ASSIGN ARCHETYPES (BASELINE: 0.5 SPLIT)
# =============================================================================

def assign_archetype(row):
    """Assign archetype based on structure and chaos scores."""
    high_structure = row['structure_score'] >= 0.5
    high_chaos = row['chaos_score'] >= 0.5
    
    if high_structure and not high_chaos:
        return 'Stable'
    elif high_structure and high_chaos:
        return 'Complex'
    elif not high_structure and high_chaos:
        return 'Messy'
    else:
        return 'Low Signal'

scores_df['archetype'] = scores_df.apply(assign_archetype, axis=1)

# Distribution summary
print("Archetype Distribution:")
print("=" * 40)
archetype_counts = scores_df['archetype'].value_counts()
for archetype in ['Stable', 'Complex', 'Messy', 'Low Signal']:
    count = archetype_counts.get(archetype, 0)
    pct = count / len(scores_df) * 100
    print(f"  {archetype:12s}: {count:>6,} series ({pct:5.1f}%)")

In [ ]:
# Preview with archetypes
scores_df[['unique_id', 'structure_score', 'chaos_score', 'archetype']].head(10)

---

## 4. Hero Selection & Validation

To validate that our map captures real behavioral differences, we select one **hero** series per quadrant:

**Selection criteria:**
- High volume (top decile) — we want series that matter
- Near quadrant centroid — representative of the archetype
- Not degenerate (no flat lines or extreme sparsity)

### 4.1 Select Heroes

In [ ]:
# =============================================================================
# COMPUTE VOLUME PER SERIES
# =============================================================================

# Total volume (sum of sales) per series
volume_by_series = df.groupby('unique_id')['y'].sum().reset_index()
volume_by_series.columns = ['unique_id', 'total_volume']

# Merge with scores
scores_df = scores_df.merge(volume_by_series, on='unique_id', how='left')

# Identify top volume decile
volume_p90 = scores_df['total_volume'].quantile(0.90)
scores_df['high_volume'] = scores_df['total_volume'] >= volume_p90

print(f"Volume threshold (90th percentile): {volume_p90:,.0f}")
print(f"High-volume series: {scores_df['high_volume'].sum():,}")

In [ ]:
# =============================================================================
# SELECT HERO FOR EACH ARCHETYPE
# =============================================================================

# Define quadrant centroids (for distance calculation)
centroids = {
    'Stable': (0.25, 0.75),      # low chaos, high structure
    'Complex': (0.75, 0.75),     # high chaos, high structure
    'Messy': (0.75, 0.25),       # high chaos, low structure
    'Low Signal': (0.25, 0.25)   # low chaos, low structure
}

heroes = {}

for archetype, (cx, cy) in centroids.items():
    # Filter to archetype + high volume
    candidates = scores_df[
        (scores_df['archetype'] == archetype) & 
        (scores_df['high_volume'])
    ].copy()
    
    if len(candidates) == 0:
        # Fall back to any series in archetype
        candidates = scores_df[scores_df['archetype'] == archetype].copy()
    
    # Calculate distance to centroid
    candidates['dist_to_centroid'] = np.sqrt(
        (candidates['chaos_score'] - cx)**2 + 
        (candidates['structure_score'] - cy)**2
    )
    
    # Select closest to centroid
    hero_id = candidates.nsmallest(1, 'dist_to_centroid')['unique_id'].values[0]
    heroes[archetype] = hero_id

print("Selected Heroes:")
print("=" * 50)
for archetype, hero_id in heroes.items():
    hero_data = scores_df[scores_df['unique_id'] == hero_id].iloc[0]
    print(f"  {archetype:12s}: {hero_id}")
    print(f"               Structure={hero_data['structure_score']:.3f}, Chaos={hero_data['chaos_score']:.3f}")
    print(f"               Volume={hero_data['total_volume']:,.0f}")
    print()

### 4.2 Hero Time Series (2×2 Facet)

Let's visually confirm that our heroes actually look different from each other.

> **Note:** No smoothers — we show raw truth for triage.

In [ ]:
# =============================================================================
# HERO TIME SERIES: 2×2 FACET
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Hero Series by Archetype (Raw Signal)', fontsize=14, fontweight='bold')

# Define positions and colors
archetype_config = {
    'Stable': {'pos': (0, 0), 'color': '#2E86AB'},
    'Complex': {'pos': (0, 1), 'color': '#8338EC'},
    'Low Signal': {'pos': (1, 0), 'color': '#6C757D'},
    'Messy': {'pos': (1, 1), 'color': '#E94F37'}
}

for archetype, config in archetype_config.items():
    row, col = config['pos']
    ax = axes[row, col]
    color = config['color']
    
    hero_id = heroes[archetype]
    hero_data = df[df['unique_id'] == hero_id].sort_values('ds')
    hero_scores = scores_df[scores_df['unique_id'] == hero_id].iloc[0]
    
    # Plot raw time series
    ax.plot(hero_data['ds'], hero_data['y'], color=color, linewidth=1.5, alpha=0.8)
    ax.fill_between(hero_data['ds'], hero_data['y'], alpha=0.2, color=color)
    
    # Title with scores
    ax.set_title(
        f"{archetype}\n{hero_id}\n"
        f"Structure={hero_scores['structure_score']:.2f}, Chaos={hero_scores['chaos_score']:.2f}",
        fontsize=10, color=color, fontweight='bold'
    )
    ax.set_xlabel('Date')
    ax.set_ylabel('Sales')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 4.3 Heroes on the Map

Let's plot our heroes as labeled points on the scatter map to show where they fall.

In [ ]:
# =============================================================================
# MAP WITH HERO OVERLAY
# =============================================================================

fig, ax = plt.subplots(figsize=(10, 10))

# Background scatter (all series, muted)
ax.scatter(
    scores_df['chaos_score'], 
    scores_df['structure_score'],
    alpha=0.15, s=8, c='#4A4E69', edgecolors='none'
)

# Quadrant lines
ax.axhline(y=0.5, color='#E94F37', linestyle='--', linewidth=2, alpha=0.5)
ax.axvline(x=0.5, color='#E94F37', linestyle='--', linewidth=2, alpha=0.5)

# Plot heroes with labels
hero_colors = {
    'Stable': '#2E86AB',
    'Complex': '#8338EC',
    'Messy': '#E94F37',
    'Low Signal': '#6C757D'
}

for archetype, hero_id in heroes.items():
    hero_data = scores_df[scores_df['unique_id'] == hero_id].iloc[0]
    color = hero_colors[archetype]
    
    # Hero point
    ax.scatter(
        hero_data['chaos_score'], 
        hero_data['structure_score'],
        s=200, c=color, edgecolors='white', linewidths=2, zorder=10,
        marker='*'
    )
    
    # Label
    ax.annotate(
        f"{archetype}\n{hero_id[:20]}...",
        xy=(hero_data['chaos_score'], hero_data['structure_score']),
        xytext=(10, 10), textcoords='offset points',
        fontsize=8, color=color, fontweight='bold',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8),
        arrowprops=dict(arrowstyle='->', color=color, lw=1)
    )

# Quadrant labels (background)
label_style = dict(fontsize=16, fontweight='bold', alpha=0.15)
ax.text(0.25, 0.75, 'STABLE', ha='center', va='center', color='#2E86AB', **label_style)
ax.text(0.75, 0.75, 'COMPLEX', ha='center', va='center', color='#8338EC', **label_style)
ax.text(0.75, 0.25, 'MESSY', ha='center', va='center', color='#E94F37', **label_style)
ax.text(0.25, 0.25, 'LOW SIGNAL', ha='center', va='center', color='#6C757D', **label_style)

ax.set_xlabel('Chaos Score →', fontsize=12)
ax.set_ylabel('Structure Score →', fontsize=12)
ax.set_title('Portfolio Map with Hero Series', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

---

## 5. Portfolio Composition (Wallet View)

Now let's understand the portfolio breakdown:
- What % of SKUs fall into each archetype?
- What % of **volume** (revenue) falls into each archetype?

These can be very different — a few high-volume SKUs might dominate revenue even if most SKUs are in a different quadrant.

### 5.1 SKU Count by Archetype

In [ ]:
# =============================================================================
# SKU COUNT BY ARCHETYPE
# =============================================================================

archetype_order = ['Stable', 'Complex', 'Messy', 'Low Signal']
archetype_colors = ['#2E86AB', '#8338EC', '#E94F37', '#6C757D']

sku_counts = scores_df['archetype'].value_counts().reindex(archetype_order)
sku_pcts = sku_counts / sku_counts.sum() * 100

fig, ax = plt.subplots(figsize=(10, 5))

bars = ax.barh(archetype_order, sku_pcts, color=archetype_colors, edgecolor='white', height=0.6)

# Add percentage labels
for bar, pct, count in zip(bars, sku_pcts, sku_counts):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
            f'{pct:.1f}% ({count:,})', va='center', fontsize=11)

ax.set_xlabel('% of SKUs', fontsize=12)
ax.set_title('Portfolio Composition: SKU Count by Archetype', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(sku_pcts) + 15)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

### 5.2 Volume by Archetype

In [ ]:
# =============================================================================
# VOLUME BY ARCHETYPE
# =============================================================================

volume_by_archetype = scores_df.groupby('archetype')['total_volume'].sum().reindex(archetype_order)
volume_pcts = volume_by_archetype / volume_by_archetype.sum() * 100

fig, ax = plt.subplots(figsize=(10, 5))

bars = ax.barh(archetype_order, volume_pcts, color=archetype_colors, edgecolor='white', height=0.6)

# Add percentage labels
for bar, pct, vol in zip(bars, volume_pcts, volume_by_archetype):
    ax.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
            f'{pct:.1f}% ({vol/1e6:.1f}M units)', va='center', fontsize=11)

ax.set_xlabel('% of Total Volume', fontsize=12)
ax.set_title('Portfolio Composition: Volume by Archetype', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(volume_pcts) + 20)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# COMPARISON: SKU COUNT vs VOLUME
# =============================================================================

comparison_df = pd.DataFrame({
    'Archetype': archetype_order,
    '% SKUs': sku_pcts.values,
    '% Volume': volume_pcts.values
})
comparison_df['Concentration'] = comparison_df['% Volume'] / comparison_df['% SKUs']

print("SKU Count vs Volume Comparison:")
print("=" * 60)
print(comparison_df.to_string(index=False))
print("\nConcentration > 1 means archetype punches above its weight in volume.")

---

## 6. The Risk Matrix (Payoff Plot)

The strategic payoff: **Where does revenue concentrate across Departments and Archetypes?**

This matrix tells us:
- Which departments are mostly Stable (→ automate)
- Which departments have Messy/Complex concentration (→ human oversight, advanced models)
- Where forecasting investment should focus

### 6.1 Build the Matrix

In [ ]:
# =============================================================================
# EXTRACT DEPARTMENT FROM UNIQUE_ID
# =============================================================================

# M5 unique_id format: CATEGORY_DEPT_ITEM_STORE (e.g., FOODS_1_001_CA_1)
# Department is the combination of CATEGORY_DEPT (e.g., FOODS_1, HOBBIES_2)

scores_df['department'] = scores_df['unique_id'].apply(
    lambda x: '_'.join(x.split('_')[:2])
)

print("Departments found:")
print(scores_df['department'].value_counts())

In [ ]:
# =============================================================================
# BUILD DEPARTMENT × ARCHETYPE MATRIX
# =============================================================================

# Aggregate volume by department and archetype
risk_matrix = scores_df.groupby(['department', 'archetype'])['total_volume'].sum().unstack(fill_value=0)

# Reorder columns
risk_matrix = risk_matrix[archetype_order]

# Convert to percentages (row-wise: within each department)
risk_matrix_pct = risk_matrix.div(risk_matrix.sum(axis=1), axis=0) * 100

print("Department × Archetype Volume Distribution (%):")
risk_matrix_pct.round(1)

In [ ]:
# =============================================================================
# OPTION A: STACKED BAR CHART
# =============================================================================

fig, ax = plt.subplots(figsize=(12, 6))

# Sort departments by total volume for meaningful ordering
dept_order = risk_matrix.sum(axis=1).sort_values(ascending=True).index
risk_matrix_sorted = risk_matrix_pct.reindex(dept_order)

# Create stacked bar
left = np.zeros(len(dept_order))
for archetype, color in zip(archetype_order, archetype_colors):
    values = risk_matrix_sorted[archetype].values
    ax.barh(dept_order, values, left=left, label=archetype, color=color, height=0.7)
    left += values

ax.set_xlabel('% of Department Volume', fontsize=12)
ax.set_ylabel('Department', fontsize=12)
ax.set_title('Risk Matrix: Volume Distribution by Department × Archetype', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=10)
ax.set_xlim(0, 100)

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# OPTION B: HEATMAP
# =============================================================================

fig, ax = plt.subplots(figsize=(10, 8))

# Sort by total volume
dept_order = risk_matrix.sum(axis=1).sort_values(ascending=False).index
heatmap_data = risk_matrix_pct.reindex(dept_order)

# Create heatmap
im = ax.imshow(heatmap_data.values, cmap='YlOrRd', aspect='auto')

# Set ticks
ax.set_xticks(range(len(archetype_order)))
ax.set_xticklabels(archetype_order, fontsize=11)
ax.set_yticks(range(len(dept_order)))
ax.set_yticklabels(dept_order, fontsize=11)

# Add text annotations
for i in range(len(dept_order)):
    for j in range(len(archetype_order)):
        value = heatmap_data.values[i, j]
        text_color = 'white' if value > 50 else 'black'
        ax.text(j, i, f'{value:.0f}%', ha='center', va='center', 
                color=text_color, fontsize=10, fontweight='bold')

ax.set_xlabel('Archetype', fontsize=12)
ax.set_ylabel('Department', fontsize=12)
ax.set_title('Risk Matrix Heatmap: % Volume by Department × Archetype', fontsize=14, fontweight='bold')

# Colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('% of Department Volume', fontsize=10)

plt.tight_layout()
plt.show()

### 6.2 Strategic Callouts

Let's extract actionable insights from the risk matrix.

In [ ]:
# =============================================================================
# STRATEGIC INSIGHTS
# =============================================================================

print("Strategic Insights by Department:")
print("=" * 70)

for dept in risk_matrix_pct.index:
    row = risk_matrix_pct.loc[dept]
    dominant = row.idxmax()
    dominant_pct = row.max()
    
    # Determine recommendation
    stable_pct = row['Stable']
    messy_pct = row['Messy']
    complex_pct = row['Complex']
    
    if stable_pct >= 60:
        recommendation = "→ AUTOMATE: High stability, simple models suffice"
    elif messy_pct >= 40:
        recommendation = "→ GOVERNANCE: High chaos, needs human-in-loop oversight"
    elif complex_pct >= 40:
        recommendation = "→ INVEST: High complexity, advanced models can capture signal"
    else:
        recommendation = "→ MIXED: Differentiated approach needed"
    
    print(f"\n{dept}:")
    print(f"  Dominant: {dominant} ({dominant_pct:.0f}%)")
    print(f"  Breakdown: Stable={stable_pct:.0f}%, Complex={complex_pct:.0f}%, Messy={messy_pct:.0f}%")
    print(f"  {recommendation}")

---

## 7. Department Context (Optional)

For scale context, we can view aggregated department-level time series.

> ⚠️ **GUARDRAIL:** Aggregation hides chaos. These plots show **scale**, not **classification**. Do not use aggregated patterns to infer archetype — that's what SKU-level scores are for.

In [ ]:
# =============================================================================
# AGGREGATED DEPARTMENT TIME SERIES
# =============================================================================

# Extract department from raw data
df['department'] = df['unique_id'].apply(lambda x: '_'.join(x.split('_')[:2]))

# Get top departments by volume
top_depts = df.groupby('department')['y'].sum().nlargest(5).index.tolist()

print(f"Top 5 departments by volume: {top_depts}")

In [ ]:
# =============================================================================
# PLOT AGGREGATED SERIES
# =============================================================================

fig, axes = plt.subplots(len(top_depts), 1, figsize=(14, 3*len(top_depts)), sharex=True)
fig.suptitle('Department-Level Aggregated Demand (Scale Context Only)', 
             fontsize=14, fontweight='bold', y=1.02)

for idx, dept in enumerate(top_depts):
    ax = axes[idx]
    
    # Aggregate to department level
    dept_data = df[df['department'] == dept].groupby('ds')['y'].sum().reset_index()
    
    # Get archetype breakdown for this department
    dept_breakdown = risk_matrix_pct.loc[dept]
    
    # Plot
    ax.plot(dept_data['ds'], dept_data['y'], color='#4A4E69', linewidth=1.2)
    ax.fill_between(dept_data['ds'], dept_data['y'], alpha=0.2, color='#4A4E69')
    
    # Title with archetype breakdown
    breakdown_str = ', '.join([f"{a}={v:.0f}%" for a, v in dept_breakdown.items() if v > 5])
    ax.set_title(f"{dept} | {breakdown_str}", fontsize=11, loc='left')
    ax.set_ylabel('Units')

axes[-1].set_xlabel('Date')

# Add guardrail note
fig.text(0.5, -0.02, 
         '⚠️ Aggregation hides SKU-level chaos. This view shows scale, not forecastability.',
         ha='center', fontsize=10, color='#E94F37', style='italic')

plt.tight_layout()
plt.show()

---

## 8. Key Takeaways

### What the Map Revealed

| Finding | Implication |
|---------|-------------|
| Portfolio composition by archetype | [Fill in: e.g., "60% of SKUs are Stable"] |
| Volume concentration | [Fill in: e.g., "80% of revenue is in Stable+Complex"] |
| Department risk profiles | [Fill in: e.g., "FOODS is mostly stable; HOBBIES is mostly messy"] |

### Strategic Recommendations by Archetype

| Archetype | % Volume | Recommended Approach |
|-----------|----------|----------------------|
| **Stable** | XX% | Automate with simple models (ETS, Prophet) |
| **Complex** | XX% | Invest in advanced models (ML, deep learning) |
| **Messy** | XX% | Human-in-loop governance, sanity checks |
| **Low Signal** | XX% | Simple baselines, don't overfit |

In [ ]:
# =============================================================================
# SAVE FINAL OUTPUT
# =============================================================================

# Save scores with archetypes for downstream use
output_cols = ['unique_id', 'department', 'structure_score', 'chaos_score', 
               'archetype', 'total_volume']
final_output = scores_df[output_cols].copy()

artifacts.save(
    final_output,
    name='1.12',
    description='Portfolio architecture: SKU archetypes and scores'
)

print(f"✓ Saved artifact '1.12' with {len(final_output):,} series")
print(f"  Columns: {list(final_output.columns)}")

---

## Next Steps

With our portfolio mapped and triaged, we can now:

1. **Build archetype-specific forecasting strategies** — different models for different behaviors
2. **Set appropriate accuracy expectations** — Stable series should hit <10% MAPE; Messy series might be 30%+
3. **Allocate resources intelligently** — invest modeling effort where it can actually help
4. **Design monitoring dashboards** — track performance by archetype, catch degradation early

> **The map is the strategy.** Every forecasting decision should flow from this triage.